In [1]:
import pandas as pd
from sodapy import Socrata
import os
from modules import data_management as dm
import glob
import zipfile
%load_ext autoreload
%autoreload 2

# Download source files into your 'local' folder.

In [ ]:
dm.download_file_maps()
dm.download_files_raw_dengue()
dm.download_files_processed_dengue()
dm.download_file_population()
dm.download_file_year()
dm.download_file_departments()

dm.download_rutinarias()

In [ ]:
dm.download_file_sstoi()
dm.download_file_tni()
dm.download_file_oni()
dm.download_file_meiv2()
dm.download_file_car()
dm.download_file_nta()

# Step by step download of climate data from two platforms...

## First, set the path to the folder where data will be stored

In [2]:
setup = "repo" #alternativo: local

try:
    if setup == "repo":
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1]) + "/local/data/"
        #data_folder = "./local/data/"
    else:
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1])  + "/data/"
        
except:
    print("Couldn't assess path automatically, setting up path locally...")
    data_folder = "D:/MyFiles/Research_Work/cw-one-health/all-dengue/dengue-project/local/data/"
    
dm.make_folder_if_not_exists(data_folder)
data_folder

Couldn't assess path automatically, setting up path locally...


'D:/MyFiles/Research_Work/cw-one-health/all-dengue/dengue-project/local/data/'

## Choose the city and retrieve all relevant data from params.csv

Amazonas
Antioquia
Arauca
Archipiélago de San Andres, Providencia y Santa Catalina
Atlantico
Bogotá
Bolivar
Boyacá
Caldas
Caquetá
Casanare
Cauca
Cesar
Chocó
Córdoba
Cundinamarca
Guainía
Guaviare
Huila
La Guajira
Magdalena
Meta
Nariño
Norte de Santander
Putumayo
Quindío
Risaralda
Santander
Sucre
Tolima
Valle del Cauca

In [4]:
df_cities = dm.read_cities_file()
df_cities

,department,municipality,code,dl_folder,starting_date,stations_temp,stations_hum,stations_prec,stations_rad,range_temp,range_hum
0,Antioquia,Medellín,05001,MEDELLIN_DATA,2011/05/20,"[27015330, 27015310, 27015300]","[27015330, 27015310, 27015300]","[27015330, 27015290, 27011270]",[],"[15.5, 37.5]","[15, 99.99]"
1,Magdalena,Santa Marta,47001,SANTA_MARTA_DATA,2008/06/28,[15015120],[15015120],"[15015050, 15010501, 15015120]",[],"[17.5, 40]","[40, 99.99]"
2,Santander,Bucaramanga,68001,BUCARAMANGA_DATA,2014/12/11,[23195230],[23195230],[23195230],[],"[17, 40]","[30, 99.99]"
3,Atlántico,Barranquilla,08001,SOLEDAD_BARRANQUILLA_DATA,2013/05/03,"[29045190, 29004520]",[29045190],"[29045120, 29040450, 29045190]",[],"[22, 45]","[45, 99.99]"
4,Atlántico,Soledad,08758,SOLEDAD_BARRANQUILLA_DATA,2007/01/01,[],[],[],[],[],[]
5,Norte de Santander,Cúcuta,54001,CUCUTA_DATA,2006-09-03,"[16015110, 16015501]","[16015110, 16015501]","[16015110, 16015010, 16015501]",[],"[17.5, 45]","[30, 99.99]"
6,Valle del Cauca,Cali,76001,CALI_DATA,2006/11/24,"[26085170, 26055120]","[26085170, 26055120]","[26055120, 26080310, 26085170]",[],"[16.5, 45]","[30, 99.99]"
7,Valle del Cauca,Palmira,76563,CALI_DATA,2006/11/24,[],[],[],[],[],[]
8,Caquetá,Villavicencio,50001,VILLAVICENCIO_DATA,2007/03/18,"[35035100, 35025110]","[35035100, 35025110]","[35030380, 35030370, 35030100, 35035020]",[],"[20, 45]","[35, 99.99]"
9,Tolima,Ibagué,73001,IBAGUE_DATA,2007/01/01,[],[],[],[],[],[]


In [19]:
# datos listos 2021: MEDELLIN, SANTA MARTA, LETICIA, PUERTO NARIÑO, CUCUTA
row_id = 8
params = {}
params_to_get = ["department","municipality","dl_folder"]
for param in params_to_get:
    params[param] = df_cities.loc[row_id,param]
params

{'department': 'Caquetá',
 'municipality': 'Villavicencio',
 'dl_folder': 'VILLAVICENCIO_DATA'}

In [20]:
dl_folder = data_folder + "src_per_city/" + params["dl_folder"]+"/"
dl_folder = dl_folder.replace("/", "\\")
dm.make_folder_if_not_exists(dl_folder)
dl_folder

'D:\\MyFiles\\Research_Work\\cw-one-health\\all-dengue\\dengue-project\\local\\data\\src_per_city\\VILLAVICENCIO_DATA\\'

## Download data from IDEAM's map using Selenium

The IDEAM API only allows downloads of these variables for up to 5 consecutive years, so we divide the relevant interval in 5 year segments.

Our relevant interval is 2006 to 2020. However, we also take years 2005 and 2021 for further review of the data.

In [21]:
variables = ["PRECIPITACION","HUM RELATIVA","TEMPERATURA",] #"RAD SOLAR"]
date_intervals = [#['01/01/2005','31/12/2009'],
                    #['01/01/2010','31/12/2014'],
                  #['01/01/2015','31/12/2019'],
                    ['01/01/2020','31/12/2022']
                ]
date_intervals

[['01/01/2020', '31/12/2022']]

In [22]:
for interval in date_intervals:
    for variable in variables:
        res = dm.download_data_with_selenium(params["department"], params["municipality"], variable, interval, dl_folder)
        if res == True:
            print("Started download for:",params["department"],"/",params["municipality"],"/",variable,interval)
            try:
                os.rename(dl_folder+"datos.zip",dl_folder+"{}.zip".format(params["department"]+"-"+params["municipality"]+"-"+variable+"-"+interval[0].replace("/","-")+"-"+interval[1].replace("/","-")))
            except Exception as e:
                print(e)
                print("Error downloading, driver closed prematurely.")
        else:
            print("Couldn't find data for:",params["department"],"/",params["municipality"],"/",variable,interval)

Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Caquetá / Villavicencio / PRECIPITACION ['01/01/2020', '31/12/2022']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Couldn't find data for: Caquetá / Villavicencio / HUM RELATIVA ['01/01/2020', '31/12/2022']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Couldn't find data for: Caquetá / Villavicencio / TEMPERATURA ['01/01/2020', '31/12/2022']


In [23]:
zipped_csvs = glob.glob(dl_folder+"*.zip")
files_count = 0
for zipped_csv in zipped_csvs:
    filename = zipped_csv.split("\\")[-1].split(".")[0]
    zipdata = zipfile.ZipFile(zipped_csv)
    zipdata.extract("excel.csv.csv", path = dl_folder)
    try:
        os.rename(dl_folder+"excel.csv.csv",dl_folder+"{}.csv".format(filename))
    except:
        print("File exists already")
    files_count+=1
    zipdata.close()

File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already
File exists already


# DOWNLOAD FROM DATOS ABIERTOS API


In [24]:
# Example authenticated client (needed for non-public datasets):
client = Socrata('www.datos.gov.co', "mc0EDr4vP5YMKFeiaAEHZUWX0")
Precipitacion = 's54a-sgyg'
Humedad = 'uext-mhny'
Temperatura = 'sbwg-7ju4'
Viento = 'sgfv-3yp8'

# PRECIPITATION DATA DOWNLOAD

In [25]:
prec_stations = dm.get_stations_from_municipality(client, Precipitacion, params["municipality"].upper())
prec_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConne

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0035035100,0240,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,Precipitacion,mm
1,3502700090,0240,GUAYURIBA - AUT,3.958169444,-73.295536111,Precipitacion,mm
2,0035025110,0240,LA LIBERTAD - AUT,4.057361111,-73.46791667,Precipitacion,mm
3,0035027140,0240,PUENTE CARRETERA - AUT,4.070194444,-73.7625,Precipitacion,mm
4,0035037100,0240,PUENTE ABADIA - AUT,4.235333333,-73.63536111,Precipitacion,mm


In [26]:
codigos_estaciones = prec_stations["CodigoEstacion"].values
prec_time_series = []
for codigo in codigos_estaciones:
    precipitation_base_data = dm.get_time_series_from_station(client, Precipitacion,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000)
    precipitation_data = dm.group_variable(precipitation_base_data,'daily')
    prec_time_series.append(precipitation_data)
all_prec_series = pd.concat(prec_time_series,axis = 0).reset_index(drop=True)
all_prec_series = dm.API_to_IDEAM_series(all_prec_series)
all_prec_series.head(2)

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,35035100,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,,mm,,META,META,VILLAVICENCIO,...,,,Precipitacion,,,2016-12-31T00:00:00.000,0.0,,,
1,35035100,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,,mm,,META,META,VILLAVICENCIO,...,,,Precipitacion,,,2017-01-01T00:00:00.000,1.6,,,


In [27]:
all_prec_series.to_csv(dl_folder+"zAPI_{}_precipitacion.csv".format(params["municipality"]), index = False)

# TEMPERATURE DATA DOWNLOAD

In [28]:
temp_stations = dm.get_stations_from_municipality(client, Temperatura, params["municipality"].upper())
temp_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0035025110,0068,LA LIBERTAD - AUT,4.057361111,-73.46791667,Temp Aire 2 m,°C
1,0035035100,0068,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,Temp Aire 2 m,°C


In [29]:
codigos_estaciones = temp_stations["CodigoEstacion"].values
temp_time_series = []
for codigo in codigos_estaciones:
    temperature_data = dm.group_variable(dm.get_time_series_from_station(client, Temperatura,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000), 'hourly')
    temp_time_series.append(temperature_data)
all_temp_series = pd.concat(temp_time_series,axis = 0).reset_index(drop=True)
all_temp_series = dm.API_to_IDEAM_series(all_temp_series)
all_temp_series.to_csv(dl_folder+"/zAPI_{}_temperature.csv".format(params["municipality"]), index = False)

# HUMEDAD DATA DOWNLOAD

In [30]:
hum_stations = dm.get_stations_from_municipality(client, Humedad, params["municipality"].upper())
hum_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0035025110,0027,LA LIBERTAD - AUT,4.057361111,-73.46791667,Humedad del aire 2 mt,%
1,0035035100,0027,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,Humedad del aire 2 mt,%


In [31]:
codigos_estaciones = hum_stations["CodigoEstacion"].values
hum_time_series = []
for codigo in codigos_estaciones:
    humedad_data = dm.group_variable(dm.get_time_series_from_station(client, Humedad,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    hum_time_series.append(humedad_data)
hum_time_series = pd.concat(hum_time_series,axis = 0).reset_index(drop=True)
hum_time_series = dm.API_to_IDEAM_series(hum_time_series)
hum_time_series.to_csv(dl_folder+"/zAPI_{}_humidity.csv".format(params["municipality"]), index = False)

# WIND DATA DOWNLOAD

In [32]:
win_stations = dm.get_stations_from_municipality(client, Viento, params["municipality"].upper())
win_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConne

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0035025110,0103,LA LIBERTAD - AUT,4.057361111,-73.46791667,Velocidad Viento (10 min),m/s
1,0035035100,0103,ICA VILLAVICENCIO - AUT,4.137388889,-73.625,Velocidad Viento (10 min),m/s


In [33]:
codigos_estaciones = win_stations["CodigoEstacion"].values
win_time_series = []
for codigo in codigos_estaciones:
    wind_data = dm.group_variable(dm.get_time_series_from_station(client, Viento,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    win_time_series.append(wind_data)
win_time_series = pd.concat(win_time_series,axis = 0).reset_index(drop=True)
win_time_series = dm.API_to_IDEAM_series(win_time_series)
win_time_series.to_csv(dl_folder+"/zAPI_{}_wind.csv".format(params["municipality"]), index = False)

In [ ]:
#Downloads from IDEAM with Selenium
variables = ["VEL VIENTO"]
#variables = ["RAD SOLAR"]
#variables = ["RAD SOLAR"]
date_intervals = [ [["01/01/{}".format(year),"31/01/{}".format(year)],
                    ["01/02/{}".format(year),"29/02/{}".format(year)],
                    ["01/03/{}".format(year),"31/03/{}".format(year)],
                    ["01/04/{}".format(year),"30/04/{}".format(year)],
                    ["01/05/{}".format(year),"31/05/{}".format(year)],
                    ["01/06/{}".format(year),"30/06/{}".format(year)],
                    ["01/07/{}".format(year),"31/07/{}".format(year)],
                    ["01/08/{}".format(year),"31/08/{}".format(year)],
                    ["01/09/{}".format(year),"30/09/{}".format(year)],
                    ["01/10/{}".format(year),"31/10/{}".format(year)],
                    ["01/11/{}".format(year),"30/11/{}".format(year)],
                    ["01/12/{}".format(year),"31/12/{}".format(year)]] 
                  for year in range(2007,2021)]
date_intervals = [di for di_tup in date_intervals for di in di_tup]